# 2D Heat Equation Simulation

Simulates the 2D heat diffusion equation using FTCS (Forward-Time Central-Space) method.

Equation: $\frac{\partial u}{\partial t} = \alpha \nabla^2 u = \alpha \left(\frac{\partial^2 u}{\partial x^2} + \frac{\partial^2 u}{\partial y^2}\right)$

- Boundary conditions: Neumann (zero flux)
- Heat source: Continuous source in middle-left region

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
from tqdm.notebook import tqdm

## Configuration Parameters

In [ ]:
# Grid dimensions
NX = 64  # Number of grid points in x-direction
NY = 64  # Number of grid points in y-direction

# Physical parameters
ALPHA = 0.01  # Thermal diffusivity (controls rate of heat spreading)
DT = 0.01     # Time step size \Delta t

# Simulation parameters
NUM_STEPS = 1000  # Total number of time steps to simulate

# Heat source parameters
SOURCE_TEMP = 100.0  # Temperature of the continuous heat source
SOURCE_SIZE = 5      # Size of heat source region (in grid cells)
SOURCE_X = NY // 2   # x-coordinate of heat source center
SOURCE_Y = NX // 4   # y-coordinate of heat source center (left quarter)

## Initialize Grid

In [ ]:
u = np.zeros((NX, NY))
u_new = np.zeros((NX, NY))

## FTCS Time Step Function with Neumann Boundary Conditions

The FTCS method uses the following finite difference approximations:

**Time derivative (Forward difference):**

$$\frac{\partial u}{\partial t} \approx \frac{u_{i,j}^{n+1} - u_{i,j}^{n}}{\Delta t}$$

**Spatial derivatives (Central difference):**

$$\frac{\partial^2 u}{\partial x^2} \approx \frac{u_{i+1,j}^{n} - 2u_{i,j}^{n} + u_{i-1,j}^{n}}{\Delta x^2}$$

$$\frac{\partial^2 u}{\partial y^2} \approx \frac{u_{i,j+1}^{n} - 2u_{i,j}^{n} + u_{i,j-1}^{n}}{\Delta y^2}$$

**Update formula:**

$$u_{i,j}^{n+1} = u_{i,j}^{n} + r_x \left(u_{i+1,j}^{n} - 2u_{i,j}^{n} + u_{i-1,j}^{n}\right) + r_y \left(u_{i,j+1}^{n} - 2u_{i,j}^{n} + u_{i,j-1}^{n}\right)$$

where $r_x = \frac{\alpha \Delta t}{\Delta x^2}$ and $r_y = \frac{\alpha \Delta t}{\Delta y^2}$

**Stability condition:** $r_x + r_y < 0.5$

In [ ]:
def ftcs_step(u, u_new, alpha, dt, dx=1.0, dy=1.0):
    nx, ny = u.shape
    
    rx = alpha * dt / (dx * dx)
    ry = alpha * dt / (dy * dy)
    
    for i in range(nx):
        for j in range(ny):
            u_xx = 0.0
            u_yy = 0.0
            
            if i == 0:
                u_xx = u[i+1, j] - u[i, j]
            elif i == nx - 1:
                u_xx = u[i-1, j] - u[i, j]
            else:
                u_xx = u[i+1, j] - 2*u[i, j] + u[i-1, j]
            
            if j == 0:
                u_yy = u[i, j+1] - u[i, j]
            elif j == ny - 1:
                u_yy = u[i, j-1] - u[i, j]
            else:
                u_yy = u[i, j+1] - 2*u[i, j] + u[i, j-1]
            
            u_new[i, j] = u[i, j] + rx * u_xx + ry * u_yy
    
    return u_new

## Apply Continuous Heat Source

In [ ]:
def apply_heat_source(u, source_temp, source_x, source_y, source_size):
    half_size = source_size // 2
    x_start = max(0, source_x - half_size)
    x_end = min(u.shape[0], source_x + half_size + 1)
    y_start = max(0, source_y - half_size)
    y_end = min(u.shape[1], source_y + half_size + 1)
    
    u[x_start:x_end, y_start:y_end] = source_temp

## Run Simulation

In [ ]:
snapshots = []
snapshot_times = [0, NUM_STEPS//4, NUM_STEPS//2, 3*NUM_STEPS//4, NUM_STEPS-1]

snapshots.append(u.copy())

for step in tqdm(range(NUM_STEPS), desc="Running simulation"):
    u_new = ftcs_step(u, u_new, ALPHA, DT)
    
    apply_heat_source(u_new, SOURCE_TEMP, SOURCE_X, SOURCE_Y, SOURCE_SIZE)
    
    u, u_new = u_new, u
    
    if step in snapshot_times:
        snapshots.append(u.copy())

print(f"Simulation complete: {NUM_STEPS} steps")
print(f"Stability criterion (r_x + r_y < 0.5): {ALPHA * DT * 2:.4f}")

## Snapshot Visualization

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

times = [0] + snapshot_times

for idx, (ax, snapshot, time) in enumerate(zip(axes, snapshots, times)):
    im = ax.imshow(snapshot, cmap='hot', origin='lower', vmin=0, vmax=SOURCE_TEMP)
    ax.set_title(f'Time step: {time}')
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    plt.colorbar(im, ax=ax, label='Temperature')

plt.tight_layout()
plt.show()

## Animation

In [ ]:
u_anim = np.zeros((NX, NY))
u_new_anim = np.zeros((NX, NY))

frames = []
frame_interval = max(1, NUM_STEPS // 200)

for step in tqdm(range(NUM_STEPS), desc="Generating animation frames"):
    u_new_anim = ftcs_step(u_anim, u_new_anim, ALPHA, DT)
    apply_heat_source(u_new_anim, SOURCE_TEMP, SOURCE_X, SOURCE_Y, SOURCE_SIZE)
    u_anim, u_new_anim = u_new_anim, u_anim
    
    if step % frame_interval == 0:
        frames.append(u_anim.copy())

fig, ax = plt.subplots(figsize=(8, 8))
im = ax.imshow(frames[0], cmap='hot', origin='lower', vmin=0, vmax=SOURCE_TEMP)
ax.set_xlabel('X')
ax.set_ylabel('Y')
plt.colorbar(im, ax=ax, label='Temperature')
title = ax.set_title('Time step: 0')

def update(frame_idx):
    im.set_array(frames[frame_idx])
    title.set_text(f'Time step: {frame_idx * frame_interval}')
    return [im, title]

anim = FuncAnimation(fig, update, frames=len(frames), interval=50, blit=True)
plt.close()

HTML(anim.to_jshtml())